<a href="https://colab.research.google.com/github/harshyadav1508/DeepLearningModel_POC/blob/main/HuggingFace/2_intro_huggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install transformers
from transformers import pipeline

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("j'adore les films de wonder woman")

[{'translation_text': "I love Wonder Woman's movies"}]

# 1. Tokenizer

In [5]:
from transformers import AutoTokenizer
checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs=["my name is harsh, i love NLP",
            "i am learning NLP from hugging face courses & i love this course"]

inputs=tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")

inputs

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  2026,  2171,  2003,  8401,  1010,  1045,  2293, 17953,
         2361,   102,     0,     0,     0,     0,     0],
       [  101,  1045,  2572,  4083, 17953,  2361,  2013, 17662,  2227,
         5352,  1004,  1045,  2293,  2023,  2607,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

# 2. Model

In [9]:
from transformers import TFAutoModel
model=TFAutoModel.from_pretrained(checkpoint)
outputs=model(inputs)
outputs.last_hidden_state.shape

TensorShape([2, 16, 768])

TensorShape([2, 16, 768]) <br> 2-> batchSize<br> 16 -> seq_length <br>768 -> dimension

In [10]:
from transformers import TFAutoModelForSequenceClassification
model=TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs=model(inputs)
outputs

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.2748252, -0.996699 ],
       [-4.143906 ,  4.425411 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [11]:
outputs.logits

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.2748252, -0.996699 ],
       [-4.143906 ,  4.425411 ]], dtype=float32)>

# 3.PostProcessing

In [15]:
import tensorflow as tf
prediction = tf.math.softmax(outputs.logits)
print(prediction)

tf.Tensor(
[[9.0649110e-01 9.3508936e-02]
 [1.8980625e-04 9.9981016e-01]], shape=(2, 2), dtype=float32)


In [22]:
bert_model=TFAutoModel.from_pretrained("bert-base-cased")
gpt_model=TFAutoModel.from_pretrained("gpt2")
bart_model=TFAutoModel.from_pretrained("facebook/bart-base")

print(type(bert_model),"\n")
print(type(gpt_model),"\n")
print(type(bart_model),"\n")

<class 'transformers.models.bert.modeling_tf_bert.TFBertModel'> 

<class 'transformers.models.gpt2.modeling_tf_gpt2.TFGPT2Model'> 

<class 'transformers.models.bart.modeling_tf_bart.TFBartModel'> 



In [24]:
from transformers import AutoConfig

bert_config=AutoConfig.from_pretrained("bert-base-cased")
gpt_config=AutoConfig.from_pretrained("gpt2")
bart_config=AutoConfig.from_pretrained("facebook/bart-base")

print(type(bert_config),"\n")
print(type(gpt_config),"\n")
print(type(bart_config),"\n")

<class 'transformers.models.bert.configuration_bert.BertConfig'> 

<class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'> 

<class 'transformers.models.bart.configuration_bart.BartConfig'> 



In [29]:
from transformers import BertConfig
bert_config=BertConfig.from_pretrained("bert-base-cased")
print(type(bert_config),"\n")

print((bert_config),"\n")

<class 'transformers.models.bert.configuration_bert.BertConfig'> 

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}
 



## Same architecture as bert-base-cased, but randomly initialized and can be trained from the scratch

In [31]:
from transformers import BertConfig, TFBertModel

bert_config=BertConfig.from_pretrained("bert-base-cased")
bert_model = TFBertModel(bert_config)

bert_config_updated=BertConfig.from_pretrained("bert-base-cased", num_hidden_layers=10)
bert_model_updtaed = TFBertModel(bert_config_updated)

#training. . .

#save
bert_model_updtaed.save_pretrained("my-new-bert-model")

#reload
bert_model_updtaed=TFBertModel.from_pretrained("my-new-bert-model")

#The Tokenization pipeline

In [ ]:
! pip install transformers

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer("let's try to tokenize!")

print(inputs,"\n")
print(inputs["input_ids"])


{'input_ids': [101, 2292, 1005, 1055, 3046, 2000, 19204, 4697, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 

[101, 2292, 1005, 1055, 3046, 2000, 19204, 4697, 999, 102]


Step1: Tokenize

In [4]:
tokens = tokenizer.tokenize("let's try to tokenize!")

print(tokens,"\n")

['let', "'", 's', 'try', 'to', 'token', '##ize', '!'] 



Step2: Convert tokens to ids

In [5]:
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(input_ids)

[2292, 1005, 1055, 3046, 2000, 19204, 4697, 999]


Step3: Prepare for Model

In [9]:
final_inputs = tokenizer.prepare_for_model(input_ids)
print(final_inputs,"\n")
print(final_inputs.input_ids)

{'input_ids': [101, 2292, 1005, 1055, 3046, 2000, 19204, 4697, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 

[101, 2292, 1005, 1055, 3046, 2000, 19204, 4697, 999, 102]


# Datset

In [19]:
! pip install datasets[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [20]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [26]:
type(raw_datasets)

datasets.dataset_dict.DatasetDict

In [23]:
print(raw_datasets["train"])

print(raw_datasets["train"][1])

print(raw_datasets["train"][:4])

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})
{'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .", 'label': 0, 'idx': 1}
{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .", 'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .', 'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .'], 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1

In [25]:
(raw_datasets["train"][:4])

{'sentence1': ['Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
  "Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .",
  'They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .',
  'Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .'],
 'sentence2': ['Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
  "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .",
  "On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .",
  'Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .'],
 'label': [1, 0, 1, 0],
 'idx': [0, 1, 2, 3]}

In [29]:
from transformers import AutoTokenizer
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
  return tokenizer(
      example["sentence1"], example["sentence2"],
      padding="max_length", truncation=True, max_length=128
      )

tokenized_datasets = raw_datasets.map(tokenize_function)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [30]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [31]:
tokenized_datasets = tokenized_datasets.remove_columns(["idx", "sentence1", "sentence2"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.with_format("tensorflow")
tokenized_datasets["train"]

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})